### HistFactory

A histfactory model is built as follows

 * A channel is modeled by a stack of samples (signal, backgrounds)
 * A measurement is composed of one or more channels

In [1]:
using namespace RooStats;
using namespace HistFactory;

#### Create the measurement

In [2]:
std::string inputFileName = "./input/example.root";
HistFactory::Measurement meas("meas", "meas");
meas.SetOutputFilePrefix( "./results/example_" );


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Define the name of the POI

In [3]:
meas.SetPOI("mu");

Declare constants

In [4]:
meas.AddConstantParam("alpha_syst1");
meas.AddConstantParam("Lumi");

Set luminosity and its uncertainty

In [5]:
meas.SetLumi(1.0);
meas.SetLumiRelErr(0.10);

...and some technical configuration...

In [6]:
meas.SetExportOnly(false);
meas.SetBinHigh(2);

#### Create a channel

In [7]:
HistFactory::Channel chan("channel1");
chan.SetData("data",inputFileName) ;
chan.SetStatErrorConfig(0.05, "Poisson");

Populate this channel with samples

**Signal sample** - scaled by a normalization factor $\mu$ and with a 5% systematic scale uncertainty named `syst1`

In [8]:
HistFactory::Sample sig( "signal", "signal", inputFileName ) ;
sig.AddOverallSys("syst1",0.95,1.05) ;
sig.AddNormFactor("mu",1,0,3 ) ;
chan.AddSample(sig) ;

**Background sample 1** - with a 5% systematic scale uncertainty named syst2 and with modeling of MC statistical, uncertainties

In [9]:
HistFactory::Sample background1("background1","background1",inputFileName) ;
background1.ActivateStatError("background1_statUncert",inputFileName) ;
background1.AddOverallSys("syst2",0.95,1.05) ;
chan.AddSample(background1) ;

**Background sample 2** - with a 5% systematic scale uncertainty named syst3 - and with modeling of MC statistical uncertainties

In [10]:
HistFactory::Sample background2( "background2", "background2", inputFileName );
background2.ActivateStatError();
background2.AddOverallSys("syst3",0.95,1.05);
chan.AddSample( background2) ;

Add channel to the measurementa

In [11]:
meas.AddChannel( chan );

Collect all the required templated histograms from the input file

In [12]:
meas.CollectHistograms();

Getting histogram.  InputFile ./input/example.root HistoPath  HistoName data
Opened input file: ./input/example.root: 0x7f5300509d50
Getting histogram.  InputFile ./input/example.root HistoPath  HistoName signal
Opened input file: ./input/example.root: 0x7f5300612840
Getting histogram.  InputFile ./input/example.root HistoPath  HistoName background1
Opened input file: ./input/example.root: 0x7f53004782c0
Getting histogram.  InputFile ./input/example.root HistoPath  HistoName background1_statUncert
Opened input file: ./input/example.root: 0x7f5300643fb0
Getting histogram.  InputFile ./input/example.root HistoPath  HistoName background2
Opened input file: ./input/example.root: 0x7f5300644bc0


Now build a workspace with a pdf and a modelconfig

In [ ]:
RooWorkspace* w =  MakeModelAndMeasurementFast(meas) ;

Making Model and Measurements (Fast) for measurement: meas
using lumi = 1 and lumiError = 0.1 including bins between 0 and 2
fixing the following parameters:
   alpha_syst1
   Lumi
Checking if output directory : ./results -  exists
Creating the output file: ./results/example__meas.root
Creating the table file: ./results/example__results.table
Creating the HistoToWorkspaceFactoryFast factory
Setting preprocess functions
Starting to process channel: channel1


-------------------
Starting to process channel1 channel with 1 observables
lumi str = [1,0,10]
lumi Error str = nominalLumi[1,0,2],0.1
[#1] INFO:ObjectHandling -- RooWorkspace::import(channel1) importing RooStats::HistFactory::FlexibleInterpVar::signal_channel1_epsilon
making normFactor: mu
signal_channel1 has no variation histograms 
processing hist signal
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(signal_channel1nominalDHist): fit range of variable obs_x_channel1 expanded to nearest bin boundaries: [1,2] --> [1,2]


Make some adjustments to histfactory workspace to conform to naming conventions of this set of example macros

In [ ]:
w->SetName("w") ;
w->data("obsData")->SetName("observed_data") ;

Print resulting workspace and save to file

In [ ]:
w->Print("t") ;

w->writeToFile("model.root") ;